# Accelerating End-to-End Data Science Workflows # 

## 02 - K-Means ##

**สารบัญ**
<br>
สมุดบันทึก (notebook) นี้จะใช้ K-means ที่ประมวลผลด้วย GPU เพื่อค้นหาตำแหน่งที่ดีที่สุดสำหรับคลังเก็บเสบียงช่วยเหลือทางอากาศ (humanitarian supply airdrop depots) ที่มีจำนวนจำกัด สมุดบันทึกนี้ครอบคลุมส่วนต่างๆ ดังนี้:
1. [สภาพแวดล้อม (Environment)](#Environment)
2. [โหลดข้อมูล (Load Data)](#Load-Data)
3. [การจัดกลุ่มด้วย K-Means (K-Means Clustering)](#K-Means-Clustering)
    * [แบบฝึกหัด #1 - สร้างอินสแตนซ์ `KMeans` อีกครั้ง (Exercise #1 - Make Another `KMeans` Instance)](#Exercise-#1---Make-Another-KMeans-Instance)
4. [แสดงผลกลุ่มข้อมูลด้วยภาพ (Visualize the Clusters)](#Visualize-the-Clusters)

## สภาพเเวดล้อม ##
ครั้งแรกที่เราจะนำเข้า `cuml` ซึ่งเป็นไลบรารีที่เร่งความเร็วด้วย GPU ของ RAPIDS ที่มีอัลกอริทึมแมชชีนเลิร์นนิงทั่วไปอยู่มากมาย เราจะแสดงผลลัพธ์ของงานของคุณในสมุดบันทึกนี้ด้วย ดังนั้นเราจึงนำเข้า `cuxfilter` ด้วย

In [ ]:
# DO NOT CHANGE THIS CELL
import cudf
import cuml

import cuxfilter as cxf

## โหลดข้อมูล ##
สำหรับโน้ตบุ๊กนี้ เราจะโหลดข้อมูลประชากรของสหราชอาณาจักร (UK) ที่ทำความสะอาดแล้วมาใช้อีกครั้ง ในกรณีนี้ เราไม่ได้ดูที่ข้อมูลระดับเทศมณฑลโดยเฉพาะ ดังนั้นเราจะละเว้นคอลัมน์นั้นไป และเก็บไว้เฉพาะคอลัมน์พิกัดกริดเท่านั้น

In [ ]:
# DO NOT CHANGE THIS CELL
gdf = cudf.read_csv('./data/clean_uk_pop.csv', usecols=['easting', 'northing'])
print(gdf.dtypes)
gdf.shape

In [ ]:
gdf.head()

<a name='#s2-3'></a>
## K-Means Clustering ##
อัลกอริทึมการจัดกลุ่มแบบ **K-means Clustering** ซึ่งเป็นแบบ **ไม่กำกับดูแล (unsupervised)** จะค้นหาจำนวน **จุดศูนย์กลาง (centroids)** ที่กำหนดไว้ `k` จุดในข้อมูล และจัดกลุ่มแต่ละจุดให้อยู่กับจุดศูนย์กลางที่ใกล้ที่สุด K-means มีประสิทธิภาพเมื่อทราบจำนวนกลุ่ม `k` หรือมีการประมาณค่าที่ดี (เช่น จากแบบจำลองกลไกพื้นฐานของปัญหา)

สมมติว่านอกเหนือจากการทราบการกระจายตัวของประชากร ซึ่งเราทราบอยู่แล้ว เราต้องการประมาณตำแหน่งที่ดีที่สุดในการสร้างคลังเสบียงเพื่อมนุษยธรรมจำนวนหนึ่งที่เราสามารถทำการหย่อนทางอากาศ (airdrop) และเข้าถึงประชากรได้อย่างมีประสิทธิภาพสูงสุด เราสามารถใช้ K-means โดยกำหนด `k` ให้เท่ากับจำนวนคลังเสบียงที่มีอยู่ และทำการฟิตกับตำแหน่งของประชากรเพื่อระบุตำแหน่งที่เป็นไปได้

K-means ที่เร่งความเร็วด้วย GPU นั้นใช้งานง่ายพอๆ กับ scikit-learn ที่ใช้ CPU เท่านั้น ในชุดแบบฝึกหัดนี้ คุณจะใช้มันเพื่อเพิ่มประสิทธิภาพตำแหน่งสำหรับคลังเสบียง 5 แห่ง

`cuml.KMeans()` จะเริ่มต้นอินสแตนซ์ K-means ตอนนี้ให้ใช้มันเพื่อเริ่มต้นอินสแตนซ์ K-means ที่ชื่อว่า `km` โดยส่งอาร์กิวเมนต์ที่ระบุชื่อ `n_clusters` กำหนดให้เท่ากับจำนวนที่เราต้องการคือ `5` ใช้เมธอด `km.fit` เพื่อฟิต `km` กับตำแหน่งของประชากรโดยส่งข้อมูลประชากรเข้าไป หลังจากฟิตแล้ว ให้เพิ่ม **ป้ายกำกับกลุ่ม (cluster labels)** กลับไปยัง `gdf` ในคอลัมน์ใหม่ที่ชื่อว่า `cluster` สุดท้าย คุณสามารถใช้ `km.cluster_centers_` เพื่อดูว่าอัลกอริทึมสร้างจุดศูนย์กลางทั้ง 5 จุดไว้ที่ใด

ด้านล่างนี้ เราจะฝึกอัลกอริทึม K-means clustering เพื่อค้นหา 5 กลุ่ม

In [ ]:
# DO NOT CHANGE THIS CELL
# instantaite
km = cuml.KMeans(n_clusters=5)

# fit
km.fit(gdf)

# assign cluster as new column
gdf['cluster'] = km.labels_
km.cluster_centers_

<a name='#s2-e1'></a>
### แบบฝึกหัด #1 - สร้าง KMeans อีกอัน ###
**คำแนะนำ**:

* แก้ไขเฉพาะส่วนที่ระบุ `<FIXME>` และรันเซลล์ด้านล่างเพื่อสร้าง K-means instance โดยมี 6 คลัสเตอร์
* แก้ไขเฉพาะส่วนที่ระบุ `<FIXME>` และรันเซลล์ด้านล่างเพื่อทำการ fit ข้อมูล 


In [ ]:
km = cuml.KMeans(n_clusters=<<<<FIXME>>>>)

In [ ]:
km.<<<<FIXME>>>>(gdf)
gdf['cluster'] = km.labels_
km.cluster_centers_

Click ... for solution. 

<a id='#s2-4'></a>
## แสดงภาพคลัสเตอร์ (Visualize the Clusters)  ##
เพื่อช่วยให้เราเข้าใจว่ากลุ่มข้อมูล (clusters) ตั้งอยู่ที่ใด เราจะสร้างภาพแสดงผลที่แยกกลุ่มเหล่านั้นออกจากกัน โดยใช้สามขั้นตอนเดิม

ด้านล่างนี้เราจะพล็อตกลุ่มข้อมูลด้วย cuxfilter

In [ ]:
# DO NOT CHANGE THIS CELL
# associate a data source with cuXfilter
cxf_data = cxf.DataFrame.from_dataframe(gdf)

# define charts
scatter_chart = cxf.charts.datashader.scatter(x='easting', y='northing')

# define widget using the `cluster` column for multiselect
# use the same technique to scale the scatterplot, then add a widget to let us select which cluster to look at
cluster_widget = cxf.charts.panel_widgets.multi_select('cluster')

In [ ]:
# create dashboard
dash = cxf_data.dashboard(charts=[scatter_chart],sidebar=[cluster_widget], theme=cxf.themes.dark, data_size_widget=True)

In [ ]:
dash.app()

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**เยี่ยมมาก!** ไปยัง [สมุดบันทึกถัดไป](3-03_dbscan.ipynb) กันเลย